# pygvec example run and visualization for a tokamak


In [ ]:
import os
from pathlib import Path
import contextlib
import shutil

import numpy as np
import matplotlib.pyplot as plt

# OMP number of threads for gvec run needs to be before import of gvec
os.environ["OMP_NUM_THREADS"]="2"
# needs `pip install` of gvec in virtual environment, and to be run in that environment!!!
import gvec  # using run & modifying the parameters & postprocessing



In [ ]:
@contextlib.contextmanager
def chdir(path: Path | str):
    """
    Contextmanager to change the current working directory.

    Using a context has the benefit of automatically changing back to the original directory when the context is exited, even if an exception is raised.
    """
    path = Path(path)
    old_dir = Path(os.getcwd())

    os.chdir(path)
    yield
    os.chdir(old_dir)

In [ ]:
template = "parameter.ini"

params= {}
iota_coefs=[0.523,0.625]  # a0*s + a1
pres_coefs=[-1.36878012275e-02,0.02] 
params["iota_coefs"] = "(/" + ", ".join(map(str, iota_coefs[::-1])) + "/)"
params["pres_coefs"] = "(/" + ", ".join(map(str, pres_coefs[::-1])) + "/)"
params["sgrid_nElems"] = 5
params["maxiter"] = 1000

runpath = Path(f"run_{1:02d}")
if not runpath.exists():
    runpath.mkdir()
    print(f"created run directory {runpath}")

gvec.util.adapt_parameter_file(template, runpath / "parameter.ini", **params)

with chdir(runpath):
    gvec.run("parameter.ini", stdout_path="stdout.txt")

statefile = sorted(runpath.glob("*State*.dat"))[-1]
with gvec.State(runpath / "parameter.ini", statefile) as state:
    rho = np.linspace(0, 1, 20)  # radial visualization points
    theta = np.linspace(0, 2 * np.pi, 50)  # poloidal visualization points
    ev = gvec.Evaluations(rho=rho, theta=theta, zeta=1, state=state)
    state.compute(ev, "X1", "X2", "LA","iota","p")

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,4))

R = ev.X1[:,:,0]
Z = ev.X2[:,:,0]
rho_vis = R*0 + ev.rho
theta_vis = R*0 + ev.theta
thetastar_vis = ev.LA[:,:,0] + ev.theta
p_vis = R*0 + ev.p
rho_levels_vis=np.linspace(0,1-1e-10,11)
theta_levels_vis=np.linspace(0,2*np.pi,16,endpoint=False)
c = ax.contourf(R,Z,p_vis)
fig.colorbar(c, ax=ax,label="pressure")
ax.contour(R,Z,rho_vis**2,rho_levels_vis,colors="black")
ax.contour(R,Z,thetastar_vis,theta_levels_vis,colors="red")
ax.set(xlabel="$R$", 
       ylabel="$Z$",
       title="equilibrium solution, cross-section",
       aspect="equal",
       xlim=[0.8*np.amin(R),1.2*np.amax(R)],
       ylim=[1.1*np.amin(Z),1.1*np.amax(Z)]
       )
ax.legend(handles=[plt.Line2D([0], [0], color='black', label='$\\rho^2=$const.'), plt.Line2D([0], [0], color='red', label='$\\theta^*=$const'),])

fig, ax = plt.subplots(1,2,figsize=(8,4))

ax[0].plot(ev.rho**2, ev.iota)
ax[0].set(xlabel="$\\rho^2\sim$ tor. flux", 
       title="iota profile")
ax[1].plot(ev.rho**2, ev.p)
ax[1].set(xlabel="$\\rho^2\sim$ tor. flux", 
       title="pressure profile")

R_axis = R[0,0].item()
Z_axis = Z[0,0].item()

print(f"Raxis={R_axis}, Zaxis={Z_axis}")